# Wrapping a basic library

We here aim at presenting the interactive wrapping workflow.
For the sake of simplicity, we consider a basic example of *C++* library.

First, import **AutoWIG**.

In [1]:
import autowig

Assuming that headers are located in the `'./basic'` forlder, we then parse headers with relevant compilation flags.

In [2]:
asg = autowig.AbstractSemanticGraph()
asg = autowig.parser(asg, ['./basic/src/cpp/overload.h', './basic/src/cpp/binomial.h'],
                          ['-x', 'c++', '-std=c++11'],
                          silent = True)

Since most of **AutoWIG** guidelines are respected, the `default` `controller` implementation is thus suitable.

In [3]:
autowig.controller.plugin = 'default'
asg = autowig.controller(asg)

In order to wrap the library we need to select the `boost_python_internal` `generator` implementation.

In [4]:
autowig.generator.plugin = 'boost_python_internal'
wrappers = autowig.generator(asg, module = 'basic/src/py/_module.cpp',
                                  decorator = 'basic/src/py/basic/_module.py',
                                  prefix = 'wrapper_')

The wrappers are only generated in-memory.
We therefore need to write them on the disk to complete the process.

In [5]:
wrappers.write()

Once the wrappers are written on disk, we need to compile the *Python* bindings using **SCons**.

In [6]:
shell = autowig.scons('./basic')

In [9]:
print shell.err

In file included from /home/pfernique/.miniconda2/envs/autowig/include/boost/python/detail/is_xxx.hpp:8:0,
                 from /home/pfernique/.miniconda2/envs/autowig/include/boost/python/detail/is_auto_ptr.hpp:9,
                 from /home/pfernique/.miniconda2/envs/autowig/include/boost/python/detail/copy_ctor_mutates_rhs.hpp:8,
                 from /home/pfernique/.miniconda2/envs/autowig/include/boost/python/detail/value_arg.hpp:7,
                 from /home/pfernique/.miniconda2/envs/autowig/include/boost/python/object/forward.hpp:13,
                 from /home/pfernique/.miniconda2/envs/autowig/include/boost/python/object/pointer_holder.hpp:16,
                 from /home/pfernique/.miniconda2/envs/autowig/include/boost/python/to_python_indirect.hpp:10,
                 from /home/pfernique/.miniconda2/envs/autowig/include/boost/python/converter/arg_to_python.hpp:10,
                 from /home/pfernique/.miniconda2/envs/autowig/include/boost/python/call.hpp:15,
          

In [8]:
import basic._module

ImportError: No module named basic._module